In [139]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

period1 = pd.read_csv('dataset/Period1.csv')
period2 = pd.read_csv('dataset/Period2.csv')
period1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154836 entries, 0 to 154835
Data columns (total 3 columns):
year         154836 non-null int64
source id    154836 non-null int64
target id    154836 non-null int64
dtypes: int64(3)
memory usage: 3.5 MB


In [150]:
#Find unique node
period1_node = pd.concat([period1['source id'], period1['target id']], ignore_index=True)
period1_node_unique = period1_node.loc[period1_node.duplicated() == False]
period1_node_unique = period1_node_unique.reset_index(drop=True)

#format edge to newortkx type
period1_edge = [(period1.loc[i,'source id'], period1.loc[i,'target id']) for i in range(len(period1))]
print('node:', len(period1_node_unique.values),'edge:', len(period1_edge))

node: 17028 edge: 154836


In [104]:
#Network graph
network_graph = nx.Graph()
#Add nodes
network_graph.add_nodes_from(period1_node_unique.values)
#Add edges
network_graph.add_edges_from(period1_edge)

print('node:', network_graph.number_of_nodes(),'edge:', network_graph.number_of_edges())

node: 17028 edge: 154698
